## Questions:

### thinking about optimal model parameters vs. making the model realistic:

Should generations overlap?
Should parent pairs produce 2 offspring or one?

We are doing **single-point** crossover.
We could also do **double-point** (two  segmentations) or **uniform** (each gene is independent) crossover.

We are using **probibalistic tournament selection** with 3 random competitors vying to be parents (based on fitness)
We could also use **deterministic5 tournament selection**
We could also use **fitness proportionate selection**, which is tournament selection among a whole group
We could choose fathers (or mothers) using one of these methods, and then choose from among that agent's mates

## Socioecological variables to review

Degree of synchrony
Degree of skew

Coding genes as parameters of a (e.g. gamma) dist

# Import packages



In [191]:
import random as rd
import math
import statistics
import numpy as np
import pandas as pd
from numpy.random import choice
from numpy.random import uniform
from numpy.random import normal
from numpy.random import randint
from numpy.random import permutation
from numpy import corrcoef
from numpy import flip
from numpy import around
from numpy import array as nparray
from random import choices as rdchoices
from random import uniform as rduniform
import numba as nb
import time
import scipy.stats
from scipy.stats import multivariate_normal as mvn
from sklearn.utils import shuffle
import statistics
import matplotlib.pyplot as plt
from matplotlib import pyplot as mp
from collections import Counter
from progress.bar import Bar
from scipy.stats import skewnorm
#import pandas as pd
from itertools import chain
import cProfile, pstats

# Evolving Model object

The class "evolvingModel" contains class variables and methods to initialize a simulated world with multiple groups and simulate biological evolution using a genetic algorithm. Initialization sets the model to generation 0 and instantiates **nGroups** social groups of class "group."

The ***evolve*** method of class "evolvingModel" loops through **nGenerations** generations. ***Evolve*** method process: First, the generation number is increased by 1. Then, the behavior of each group is simulated for a single generation using the ***runModel*** method of class "group." The ***setupNextGen*** and ***setGenotypes*** methods of class "group" then decides on a cohort of **nAgents** mothers and **nAgents** fathers for the next generation, perform  genetic **recombination** using mothers' and fathers' genes, performs probabilistic **mutation** of offspring genes, and finally initializes a new generation of **nFemales** and **nMales** with the those genes. Finally, the ***migration*** method of class "evolvingModel" probabilistically selects some number of the next gerenations' agents to migrate to new groups, based on parameter **migrationRate**, with a probability of **maleDispersalBias** that the agent selected for migration with be male. Migration switches the following class variables of the agent (a member of either class "female" or "male") : **"genes", "swellingList"** (females only), and **"cost"**.

If **realTimePlots** is set to True, a dot plot of average swelling strengh (Y-axis) across group 0 against cycle day (X-axis) appears in a quartz window following the first generation and updates with probability **realTimePlotsRate** for future generations. Following a model run of **nGenerations**, a line plot showing changes across generations in the intra-generation variability of cycle days on which the alpha male of group 0 mates, with lines for first quartile, median, and 3rd quartile of mating days. The Y-axis is mating days and the X-axis is generations. This concludes the model run.

Key: \
"Quotation marks" indicates a class \
**bold** indicates global variables \
***bold, italicized*** indicates methods \
**"bold, in quotation marks"** indicates class variables


In [2069]:
class evolvingModel:
    
    def __init__(self):
        
        self.generation = 0
        
        self.groups = []
        
        for g in range(nGroups):
            self.groups.append(group(g))

        self.alphaMates, self.alphaMatingSpread = [], [[],[],[]]
        
        self.alphaMates, self.alphaMatingMedian,  self.alphaMating1Q, self.alphaMating3Q = [], [], [], []
        
        self.potentialMoms = rdchoices(range(nFemales), k = 3 * nAgents * nGroups * nGenerations)
        self.potentialDads = rdchoices(range(nMales), k = 3 * nAgents * nGroups * nGenerations)
        
        self.genesInGroup = numberGenes * nAgents 
        self.mutations = [round(uniform(0,self.genesInGroup * mutationRate * 2)) for i in range(nGenerations + 1)]
        
    def evolve(self):
          
#         if realTimePlots == True:
#             %matplotlib qt
        
        while self.generation < nGenerations:
            self.generation += 1
            
            g = 0
            while g < nGroups:
                self.groups[g].runModel() 
                g += 1
        
#             if realTimePlots == True and (rd.uniform(0,1) < realTimePlotsRate or self.generation == 1):
#                 self.plotSwelling() if whichPlot == "Swelling" else self.plotPairs()
#             elif rd.uniform(0,1) > 0.99:
#                 print(self.generation)
            
            #self.updateAlphaMatingDays()
            
#             if self.generation == nGenerations - 1:
#                 %matplotlib inline
#                 self.plotRS()
               
            for g in self.groups:
                g.setupNextGen()
                g.setGenotypes()
               
            if dispersal == True:
                self.migration()
        
        lst = []
        lstLower = []
        lstUpper =[]
        for j in range(cycleLength):
            lst.append(statistics.mean([f.swellingList[j] for f in sum([g.females for g in model.groups], [])]))
            SEM = scipy.stats.tstd([f.swellingList[j] for f in sum([g.females for g in model.groups], [])])
            lstLower.append(lst[j] - SEM) if SEM < lst[j] else lstLower.append(0)
            lstUpper.append(lst[j] + SEM)

        modelData[modelRuns] = [rankFitnessCorrelation]+[synchrony]+[popSize]+ lst + [SEM] + lstLower + lstUpper
        
#         if realTimePlots == False:
#             self.plotSwelling()
#             plt.figure()
#             self.plotPairs()
            
        #self.plotMatingDays()
            
    def migration(self):
                
        migrations = round(uniform(totalAgents * migrationRate * 2))
        groupsLeavingFrom = choice([g for g in self.groups], migrations)
        agentsLeaving = choice(range(nMales), migrations)
        
        for m in range(migrations):
            
            groupLeavingFrom = groupsLeavingFrom[m]
            
            if rd.uniform(0,1) > maleDispersalBias:
                agentLeaving = groupLeavingFrom.males[agentsLeaving[m]]
                agentComing = rd.choice(rd.choice([g for g in self.groups if g != groupLeavingFrom]).males)
            else:
                agentLeaving = groupLeavingFrom.females[agentsLeaving[m]]
                agentComing = rd.choice(rd.choice([g for g in self.groups if g != groupLeavingFrom]).females)
            
            tempGenes, tempCost = agentLeaving.genes, agentLeaving.cost
            agentLeaving.genes, agentLeaving.cost = agentComing.genes, agentComing.cost
            agentComing.genes, agentComing.cost = tempGenes, tempCost
        
    def plotSwelling(self):

        lst = []
        lstLower = []
        lstUpper =[]
        for j in range(cycleLength):
            lst.append(statistics.mean([f.swellingList[j] for f in sum([g.females for g in model.groups], [])]))
            SEM = scipy.stats.tstd([f.swellingList[j] for f in sum([g.females for g in model.groups], [])])
            lstLower.append(lst[j] - SEM) if SEM < lst[j] else lstLower.append(0)
            lstUpper.append(lst[j] + SEM)

        if rd.choice([1,2]) == 1:
            plt.clf()
            plt.plot(lst, "bo")
            plt.plot(lstLower, "r")
            plt.plot(lstUpper, "r")
            plt.ylim = [0,max(lst) * 1.1]
            plt.text(0.1, max(lst) * 0.9, str(self.generation))
            plt.pause(0.000001)
            plt.show()
            
    def plotRS(self):
        
        femaleRS = [f.reproductiveSuccess for f in model.groups[0].females]
        maleRS = [m.reproductiveSuccess for m in model.groups[0].males]
        
        print("SD^2 in female RS:"+str(statistics.variance(femaleRS)))
        print("SD^2 in male RS:"+str(statistics.variance(maleRS)))

        plt.figure()
        plt.hist(maleRS, alpha=0.5, label = "male RS")
        plt.hist(femaleRS, alpha=0.5, label = "female RS")
        plt.title('Male and female Reproductive Success')
        plt.legend(loc='upper right')
        plt.show()
        
        plt.figure()
        plt.plot([m.rank for m in model.groups[0].males],
                 [m.reproductiveSuccess for m in model.groups[0].males], 'bo')
        plt.title('male RS ~ rank')
        plt.show()
        
    def plotMatingDays(self):
        
        plt.figure()
        
        for i in range(3):
            plt.plot(range(self.generation), self.alphaMatingSpread[i])
        
        plt.ylim = [0, cycleLength]
        plt.xlim = [0, 1.0]
        plt.title("Synchrony: " + str(round(synchrony, 2)) + "; Rank/Fitness Correlation: " + str((rankFitnessCorrelation , 2)))
        
    def updateAlphaMatingDays(self):
    
        self.alphaMates.extend(self.groups[0].males[0].mateTiming)
        a,b,c = np.percentile(self.alphaMates,[25, 5, 75])

        self.alphaMatingSpread[0].append(a)
        self.alphaMatingSpread[1].append(b)
        self.alphaMatingSpread[2].append(c)
        
    def plotPairs(self):
        plt.clf()
        plotGroup = model.groups[0]
        plotGroup.females = sorted(choice(plotGroup.females, size=nFemales, replace=False), key=plotGroup.sortSwelling)
        swellings = [f.swelling * 1000 for f in plotGroup.females]
        IDs = [f.ID for f in plotGroup.females]
        plt.scatter([1] * nFemales, [f.ID for f in plotGroup.females], s = swellings)
        plt.scatter([2] * nMales, [m.rank for m in plotGroup.males], s = [m.rank for m in plotGroup.males])
        plt.scatter(0,0, s = 0)
        plt.scatter(3,0, s = 0)
        mates = [[plotGroup.females[i].ID] + [plotGroup.males[i].rank] for i in range(nPairs)]
        mates = [agent for pair in mates for agent in pair]
        for i in range(nPairs):
            plt.plot([1,2],[plotGroup.females[i].ID, plotGroup.males[i].rank],linewidth=0.5)
        plt.xlim = [0,3]
        plt.text(0.1, nMales * 0.9, str(model.generation))
        plt.title('mating pairs based on female swelling size (left)\nand male rank (right)')
        plt.pause(0.00001)
        plt.show()
        

# Group object

The class "group" generates and simulates the behavior of a single population over the course of a mating season. At initialization, the **"run"** boolean variable is set to "True" and **"step"**, which keeps count of timesteps (days) of the simulation, is set to 0. The ***setFitness*** method then sets up a list of male fitness values (**"fitnessList"**) that is correlated to male ranks with a correlation coefficient of approximately **rankFitnessCorrelation **. Finally, **nFemales** objects of class "female" and **nMales** objects of class "male" are instantiated in lists (**"males"** and **"females"**) contained in the "group" object.

The ***runModel*** method simulates agent behavior for **nDays** timesteps (days). It first calls the **makeMatingPairs** method, which orders "male" and "female" objects by **"rank"** and **"swelling"**, respctively. Mating pairs are created by pairing males and females with the same index in their respective ordered lists. Males then receive an increase to their **"reproductiveSuccess"** variable in the amount of the current **"conceptionProbability"** of their mate, and females receive an increase to their **"reproductiveSuccess"** variable in the amount of their current **"conceptionProbability"** multipled by the **"fitness"** of their mate.

For each "female" object, the ***setupCycleDay*** method of class "female" is run to 1) increase **"cycleDay"** by one, and set **"swelling"** and **"conceptionProbability"** based on the unique **"swellingList"** associating that "female's" **"cycleDay"** and **"swelling"** strength variables, and the global **conceptionProbabilityList**, which associates **"cycleDay"** with **"conceptionProbability"**.

Key: \
"Quotation marks" indicates a class \
**bold** indicates global variables \
***bold, italicized*** indicates methods \
**"bold, in quotation marks"** indicates class variables

In [2037]:
nMales = 25
nFemales = 25
nAgents = nFemales + nMales
nPairs = min(nMales, nFemales)
ranks = range(nMales)
nDays = 100
rankFitnessCorrelation = 0.66
nGenerations = 1
model = evolvingModel()
profiler = cProfile.Profile()
profiler.enable()
#model.groups[0].runModel()
model.evolve()
profiler.disable()
stats = pstats.Stats(profiler).sort_stats('tottime')
stats.print_stats()

         2377263 function calls (2202435 primitive calls) in 5.048 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    14569    0.607    0.000    2.527    0.000 /Users/kevinrosenfield/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2275(cov)
   160259    0.539    0.000    0.539    0.000 {built-in method numpy.array}
189471/14643    0.482    0.000    4.231    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    14569    0.340    0.000    4.173    0.000 /Users/kevinrosenfield/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2502(corrcoef)
    14569    0.259    0.000    0.579    0.000 /Users/kevinrosenfield/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161(_mean)
    14569    0.215    0.000    0.326    0.000 /Users/kevinrosenfield/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/stride_tricks.py:339(_broadcast_to)
    29138    0.200    

In [ ]:
nDays = 60
nGenerations = 1000
modelRuns = 0 
#for popSize in [15,50,100]:
for popSize in [15]:
    nMales = popSize
    nFemales = popSize
    nAgents = nFemales + nMales
    nPairs = min(nMales, nFemales)
    ranks = range(nMales)
    #for rankCorVal in [0, 0.33, 0.66, 1.0]:
    for rankCorVal in [0.66]:
        rankFitnessCorrelation = rankCorVal
        for synchronyVal in [0, 0.33, 0.66, 1.0]:
            synchrony = synchronyVal
            model = evolvingModel()
            model.evolve()
            modelRuns += 1
            print(modelRuns)


/Users/kevinrosenfield/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:143: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


In [2088]:
modelData

,0,1,2,3,4,5,6,7
0,0.000000,0.000000,0.000000,0.0,0.33,0.330000,0.330000,0.330000
1,0.000000,0.330000,0.660000,1.0,0.00,0.330000,0.660000,1.000000
2,15.000000,15.000000,15.000000,15.0,15.00,15.000000,15.000000,15.000000
3,0.000000,0.000000,0.000294,0.0,0.00,0.000752,0.000000,0.000000
4,0.000000,0.000000,0.000196,0.0,0.00,0.001194,0.000245,0.000000
...,...,...,...,...,...,...,...,...
89,0.003509,0.000274,0.002842,0.0,0.00,0.009257,0.007565,0.001718
90,0.003790,0.000260,0.002839,0.0,0.00,0.008934,0.007023,0.001718
91,0.004078,0.000246,0.000857,0.0,0.00,0.008680,0.006481,0.001718
92,0.004370,0.000233,0.000845,0.0,0.00,0.008489,0.005949,0.001718


In [2051]:
fitnessList = ranks
i = 0.5
while abs(0 - rankFitnessCorrelation  + corrcoef(ranks, fitnessList)[1,0]) > 0.01:
    fitnessList = [fitnessList[f] + rduniform(-i,i) for f in range(len(fitnessList))]
    i += 0.05
    i = 0.5 if i == 10 else i
    print(corrcoef(ranks, fitnessList)[1,0])

0.9998029025079276
0.9995773041827937
0.9993276244496399
0.9991766075875365
0.9985829211354836
0.9983122002025189
0.9979236555049077
0.9969689708485938
0.9961037987034232
0.9954744868855726
0.9948759162168409
0.9952224002438667
0.9944042257007651
0.9944869839631691
0.9936833037166134
0.9926230365467462
0.9916728350833343
0.9909784508424038
0.9883057729013529
0.986215353509877
0.9838084543288014
0.9826376284421953
0.9826029230469033
0.9835972729133542
0.9819808389773199
0.977269433274946
0.9741229833974903
0.9734258231989078
0.9708242771064289
0.9626795076759684
0.9627169631549608
0.9586571661133784
0.951345018782033
0.9518770061798522
0.9482708806431847
0.9453200512868667
0.935630530352742
0.9395931155403096
0.9276045196176789
0.9265607823303648
0.9218176133630865
0.9185884488851682
0.9167462505350294
0.9126593402855457
0.9083476352342796
0.904489048818648
0.9035600169921418
0.9032193961711482
0.8943409955341975
0.8909655045723197
0.8824649815020369
0.8842075996541604
0.884656473780921

In [2047]:
ranks

range(0, 50)

In [2057]:
class group:
    
    def __init__(self, g):
        
        self.ID = g
        self.step = 0
        
        #start = time.time()
        self.setFitness()
        self.cycleDayList = randint(1, round((cycleLength - 1) * (1 - synchrony)) + 2, size = nFemales)
        #print(time.time() - start)
        
        self.males, self.females = [], []
        
        self.tieBreaker = uniform(0,0.00000000001, nFemales * nDays)
        
        #start = time.time()
        for m in range(nMales):
            self.males.append(Male(m, self.fitnessList[m]))

        for f in range(nFemales):
            self.females.append(Female(f, self.cycleDayList[f]))
        #print(time.time() - start)
        
    def runModel(self):

        while self.step < nDays:
                
            self.females = sorted(self.females, key=self.sortSwelling)
            self.makeMatingPairs()
            self.setupCycleDay()

            self.step += 1
            
            '''
            elif self.step == nDays / 2:
                print(self.step)
            ''' 
            
    def setupCycleDay(self):
        self.cycleDayList += 1
        self.cycleDayList[self.cycleDayList == 31] = 1
        for f in self.females:
            f.cycleDay = self.cycleDayList[f.ID]
            f.swelling = f.swellingList[f.cycleDay - 1]
            f.swelling += self.tieBreaker[nFemales * (self.step):nFemales * (self.step + 1)][f.ID]
            f.conceptionProbability = conceptionProbabilityList[f.cycleDay - 1]
    
    def sortSwelling(self, f):
        return f.swelling

    def makeMatingPairs(self):
        i = 0
        while i < nPairs:
            f = nFemales - 1 - i
            self.males[i].reproductiveSuccess += self.females[f].conceptionProbability * self.males[i].fitness
            #self.males[i].mateTiming.append(self.females[f].cycleDay)
            self.females[f].reproductiveSuccess += self.females[f].conceptionProbability * self.males[i].fitness
            i += 1
        
    def setFitness(self):

        fitnessList = ranks
        i = 0.5
        while abs(0 - rankFitnessCorrelation  + corrcoef(ranks, fitnessList)[1,0]) > 0.05:
            fitnessList = [fitnessList[f] + rduniform(-i,i) for f in range(len(fitnessList))]
            i += 0.05
            if i == 10:
                i = 0.5
                fitnessList = ranks
            
        self.fitnessList = flip((fitnessList - np.min(fitnessList))/np.ptp(fitnessList))
            
    def setupNextGen(self):
        
        self.nextGenMotherGenes = []
        motherProbabilities = [f.reproductiveSuccess - f.cost for f in self.females]
        # lack of ability to chose becomes a cost as rankFitnessCorrelation  goes down

        self.nextGenFatherGenes = []
        fatherProbabilities = [m.reproductiveSuccess for m in self.males] # does male fitness matter?
            
        parentsStartingPoint = model.generation * nGroups * nAgents + self.ID * nAgents    
        
        moms = [rdchoices(model.potentialMoms[parentsStartingPoint + i: parentsStartingPoint + i + 2],
                          weights=[motherProbabilities[p] for p in model.potentialMoms[
                              parentsStartingPoint + i: parentsStartingPoint + i+2]],k = 1)[0] for i in np.arange(0, (nAgents * 3), 3)]
        dads = [rdchoices(model.potentialDads[parentsStartingPoint + i:parentsStartingPoint + i + 2],
                          weights=[fatherProbabilities[p] for p in model.potentialDads[
                              parentsStartingPoint + i: parentsStartingPoint + i+2]], k = 1)[0] for i in np.arange(0, (nAgents * 3), 3)]
            
        self.nextGenMotherGenes = [self.females[m].genes for m in moms]
        self.nextGenFatherGenes = [self.males[d].genes for d in dads]

        self.recombination()
        self.mutation() if model.mutations[model.generation] > 0 else 0
        self.reset()
            
    def recombination(self):
        
        self.offspringGenes = []
        recombinationPoints = choice(range(numberGenes), nAgents)
        splitTypes = randint(0,2, nAgents)
        i = 0
        while i < nAgents:
            recombinationPoint = recombinationPoints[i]
            if splitTypes[i] == 1:
                self.offspringGenes.append([m for m in self.nextGenMotherGenes[i][:recombinationPoint]] + 
                                           [f for f in self.nextGenFatherGenes[i][recombinationPoint:]])
            else:
                self.offspringGenes.append([f for f in self.nextGenFatherGenes[i][:recombinationPoint]] + 
                                           [m for m in self.nextGenMotherGenes[i][recombinationPoint:]])
            i += 1
                
    def mutation(self):
        
        genesInGroup = nAgents * numberGenes
        mutations = model.mutations[model.generation]
       
        dayMutations = randint(mutations)
        peakMutations = mutations - dayMutations
      
        dayGenesMutating = choice(range(4), dayMutations)
        peakGenesMutating = [4] * (peakMutations)
        
        dayOffspringsMutating = choice(range(len(self.offspringGenes)), dayMutations)
        peakOffspringsMutating = choice(range(len(self.offspringGenes)), peakMutations)

        dayPertubations = choice([-1,1], dayMutations)
        peakPertubations = uniform(-0.02,0.02, peakMutations)
        
        newDayGenes = nparray([self.offspringGenes[dayOffspringsMutating[m]][dayGenesMutating[m]] + dayPertubations[m] for m in range(dayMutations)])
        newPeakGenes = nparray([self.offspringGenes[peakOffspringsMutating[m]][peakGenesMutating[m]] + peakPertubations[m] for m in range(peakMutations)]) 
        
        newDayGenes[newDayGenes > 30] = randint(cycleLength)
        newDayGenes[newDayGenes < 0] = randint(cycleLength)
        newPeakGenes[newPeakGenes < 0] = 0
        for m in range(dayMutations):
            self.offspringGenes[dayOffspringsMutating[m]][dayGenesMutating[m]] = newDayGenes[m]
            self.offspringGenes[m][self.offspringGenes[m].index(
                max(self.offspringGenes[m]))] = max(self.offspringGenes[m]) - sum(
                self.offspringGenes[m][:4]) - cycleLength if self.offspringGenes[m][sum(
                self.offspringGenes[m][:4]) > 30] == 2 else max(self.offspringGenes[m])
        for m in range(peakMutations):
            self.offspringGenes[peakOffspringsMutating[m]][peakGenesMutating[m]] = newPeakGenes[m]
     

    def setGenotypes(self):
        
        for f in self.females:
            f.genes = self.offspringGenes[f.ID]
            f.swellingList = f.setSwelling()
            #f.cost = sum(self.offspringGenes[f.ID])
            #f.swellingList = f.genes
            f.setupCycleDay()

        for m in self.males:
            m.genes = self.offspringGenes[m.rank + nFemales]
            
    def reset(self):
        self.step = 0
        self.setFitness()
        self.cycleDayList = randint(1, round((cycleLength - 1) * (1 - synchrony)) + 2, size = nFemales)
        self.males, self.females = [], []
        self.males = [Male(m, self.fitnessList[m]) for m in range(nMales)]
        self.females = [Female(f, self.cycleDayList[f]) for f in range(nFemales)]
            

In [1959]:
nDays = 1
model = evolvingModel()

cycles = math.ceil(nDays / cycleLength)

IDs = pd.DataFrame([[d] * nDays for d in range(nFemales)])
cycleDays = pd.DataFrame([f.cycleDay-1 for f in model.groups[0].females])
IDs2 = IDs.copy()
IDs3 = IDs.copy()

swellings = pd.DataFrame([(
    f.swellingList[cycleDays.iloc[f.ID,0]-1:30] + f.swellingList*cycles)[1:
    nDays+1] for f in model.groups[0].females])

swellings = pd.DataFrame([model.groups[0].tieBreaker[nFemales * (i-1):nFemales * i] for i in range(1,nDays+1)]).T + swellings
swellings2 = swellings.copy()
swellings3 = swellings.copy()

conceptionRisks = pd.DataFrame([(
    conceptionProbabilityList[cycleDays.iloc[f.ID,0]-1:30] + conceptionProbabilityList*cycles)[1:
    nDays+1] for f in model.groups[0].females])

fitnesses = pd.DataFrame([np.flip(model.groups[0].fitnessList) for i in range(1,nDays+1)]).T

conceptionRisks2 = conceptionRisks.copy()
conceptionRisks3 = conceptionRisks.copy()

for x in range(nDays):
    IDs2[x] = [i for _,i in sorted(zip(swellings2[x],IDs3[x]))]
    conceptionRisks2[x] = [i for _, i in sorted(zip(swellings3[x],conceptionRisks3[x]))]
    
maleReproductiveSuccess = conceptionRisks2.multiply(fitnesses)
maleReproductiveSuccess2 = pd.DataFrame(maleReproductiveSuccess.copy())

femaleReproductiveSuccess = pd.DataFrame([[i for _,i in sorted(zip(swellings3[x],maleReproductiveSuccess2[x]),reverse=True)] for x in range(nDays)]).T
maleReproductiveSuccess2 = maleReproductiveSuccess2

model.groups[0].runModel()

print(pd.DataFrame([f.reproductiveSuccess for f in model.groups[0].females],[m.reproductiveSuccess for m in model.groups[0].males]))

print(pd.concat([femaleReproductiveSuccess,maleReproductiveSuccess2], axis = 1))


                 0
0.103733  0.000000
0.000000  0.103733
     0    0
0  0.0  0.0
1  0.0  0.0


In [1945]:
nDays = 1
model = evolvingModel()

cycles = math.ceil(nDays / cycleLength)

IDs = pd.DataFrame([[d] * nDays for d in range(nFemales)])
cycleDays = pd.DataFrame([f.cycleDay-1 for f in model.groups[0].females])
IDs2 = IDs.copy()
IDs3 = IDs.copy()

swellings = pd.DataFrame([(
    f.swellingList[cycleDays.iloc[f.ID,0]-1:30] + f.swellingList*cycles)[1:
    nDays+1] for f in model.groups[0].females])

print(swellings)
print([f.swelling for f in model.groups[0].females])

   0
0  0
1  0
[0, 0]


In [1960]:
nDays = 2
model = evolvingModel()

cycles = math.ceil(nDays / cycleLength)

IDs = pd.DataFrame([[d] * nDays for d in range(nFemales)])
cycleDays = pd.DataFrame([f.cycleDay for f in model.groups[0].females])
IDs2 = IDs.copy()
IDs3 = IDs.copy()

swellings = pd.DataFrame([(
    f.swellingList[cycleDays.iloc[f.ID,0]-1:30] + f.swellingList*cycles)[1:
    nDays+1] for f in model.groups[0].females])

swellings = pd.DataFrame([model.groups[0].tieBreaker[nFemales * (i-1):nFemales * i] for i in range(1,nDays+1)]).T + swellings
swellings2 = swellings.copy()
swellings3 = swellings.copy()

conceptionRisks = pd.DataFrame([(
    conceptionProbabilityList[cycleDays.iloc[f.ID,0]-1:30] + conceptionProbabilityList*cycles)[1:
    nDays+1] for f in model.groups[0].females])

model.groups[0].setupCycleDay()

print(swellings)
print([f.swelling for f in model.groups[0].females])
print(conceptionRisks)
print([f.conceptionProbability for f in model.groups[0].females])
print(cycleDays)
print([f.cycleDay for f in model.groups[0].females])


              0             1
0  1.678752e-12  8.976650e-13
1  1.850680e-12  7.234636e-12
[1.6787520171796475e-12, 1.850679707360704e-12]
          0         1
0  0.198206  0.254082
1  0.000000  0.000000
[0.19820558, 0]
    0
0  12
1   3
[13, 4]


In [1915]:
pd.DataFrame([model.groups[0].tieBreaker[nFemales * (i-1):nFemales*i] for i in range(1,nDays+1)]).T


,0,1
0,4.972237e-12,5.850621e-12
1,1.639958e-12,2.872099e-12


In [1913]:
model.groups[0].tieBreaker

array([4.97223727e-12, 1.63995757e-12, 5.85062079e-12, 2.87209860e-12])

In [1814]:
fitnesses = pd.DataFrame([model.groups[0].fitnessList for i in range(1,nDays+1)]).T

print(fitnesses)
print([m.fitness for m in model.groups[0].males])

     0
0  1.0
1  0.0
[1.0, 0.0]


## Male object

Calling the class "male" instantiates a "male" object with the following class variables representing its behavioral and biological traits:

**"rank"** represents the agent's position in mating competition; the **makeMatingPairs** method of class "group" matches males of lower numerical **"rank"** (higher dominance) with females of higher **"swelling"**  for mating.

**"fitness"** is correlated with **"rank"** with a correlation coefficient of approximately **rankFitnessCorrelation ** and influences the chances that mating will result in surviving offspring when it is multiplied by **"conceptionRisk"** to determine **"reproductiveSuccess"**.

**"reproductiveSuccess"** results from mating with fertile females and through the **setupNextGen** method of class "group" and determines which males wil produce offspring to populate the next generation.

**"mateTiming"** indicates the **"cycleDay"** on which mating occurred (for visualiztion purposes only)

**"startingSwelling"** indicates how the fertility swelling **"genes"** will be distributed at the start of the model run. Because males do not swelling, this does not affect their behavior during th model, but can influence that of their female offspring.

**"genes"** determine swelling strength and are passed on to offspring. They only influence female behavior.

**"cost"** of swellinging only influences female reproduction.

Key: \
"Quotation marks" indicates a class \
**bold** indicates global variables \
***bold, italicized*** indicates methods \
**"bold, in quotation marks"** indicates class variables

In [793]:
class Male:
    
    __slots__ = 'rank', 'fitness', 'reproductiveSuccess',  'genes', 'cost', 'mateTiming'
    
    def __init__(self, m, fitness):
        self.rank = m
        self.fitness = fitness
        self.reproductiveSuccess = 0
        self.mateTiming = []
        if startingSwelling == "noSwelling":
            self.genes = [0] * 5
        elif startingSwelling == "randomUniform":
            self.genes = [randint(0,cycleLength+1)]
            self.genes += [randint(0,cycleLength+1 - sum(self.genes))]
            self.genes += [randint(0,cycleLength+1 - sum(self.genes))]
            self.genes += [randint(0,cycleLength+1 - sum(self.genes))] + [uniform(0,1)]
        self.cost = 0
        

# Female object

Calling the class "female" instantiates a "female" object with the following class variables representing its behavioral and biological traits:

**"ID"** helps locate the agent's index in lists

**"cycleDay"** determines the day of the cycle on which females start the model. High cycle **synchrony** constrains  it to fewer days

**"reproductiveSuccess"** results from mating with fit males and through the **setupNextGen** method of class "group" and determines which females will produce offspring to populate the next generation.

**"startingSwelling"** indicates how the fertility swelling **"genes"** will be distributed at the start of the model run.

**"genes"** determine swelling strength and are passed on to offspring

**"cost"** of swellinging is calculated by adding absolute daily swelling strengths and daily increases in swelling strength

Key: \
"Quotation marks" indicates a class \
**bold** indicates global variables \
***bold, italicized*** indicates methods \
**"bold, in quotation marks"** indicates class variables

In [2007]:
class Female:

    __slots__ ='ID','cycleDay','mateList','reproductiveSuccess','genes','cost','swellingList','swelling','conceptionProbability'
    
    def __init__(self, f, cycleDay):
        self.ID = f
        self.cycleDay = cycleDay
        self.mateList = []
        self.reproductiveSuccess = False
        if startingSwelling == "noSwelling":
            self.genes = [0] * 5
        elif startingSwelling == "randomUniform":
            self.genes = [randint(0,cycleLength+1)]
            self.genes += [randint(0,cycleLength+1 - sum(self.genes))]
            self.genes += [randint(0,cycleLength+1 - sum(self.genes))]
            self.genes += [randint(0,cycleLength+1 - sum(self.genes))] + [uniform(0,1)]
           
        self.swellingList = self.setSwelling()
        self.swelling = self.swellingList[self.cycleDay - 1]
        
        self.conceptionProbability = conceptionProbabilityList[self.cycleDay - 1]

    def setupCycleDay(self):  
        self.cycleDay = self.cycleDay + 1 if self.cycleDay < cycleLength else 1  
        self.swelling = self.swellingList[self.cycleDay - 1]
        self.conceptionProbability = conceptionProbabilityList[self.cycleDay - 1]
        
    def setSwelling(self):
        inceraseStartDay, increaseDuration, peakDuration, decreaseDuration, peakSwelling = self.genes
        peakStartDay = inceraseStartDay + increaseDuration
        decreaseStartDay = peakStartDay + peakDuration
        decreaseEndDay = decreaseStartDay + decreaseDuration
        increaseCoefficient =  peakSwelling / (increaseDuration + 1)
        decreaseCoefficient =  peakSwelling / (decreaseDuration + 1)

        x =  np.arange(1,cycleLength + 1,1)
        y = [0] * (inceraseStartDay - 1) + [0 + increaseCoefficient * i for i in range(1,increaseDuration)]
        y = y + [peakSwelling] * peakDuration + [peakSwelling - decreaseCoefficient * i for i in range(1,decreaseDuration+1)]
        y = y + [0] * (cycleLength - len(y))

        self.cost = sum(y) + (y[0]*10)
        for g in range(1, len(y)): # to add cost of growth
            self.cost = self.cost + (y[g] - y[g - 1])*10 if y[g] > y[g-1] else self.cost
            
        return(y)

In [1646]:
model.groups[0].tieBreaker[model.groups[0].step * nFemales + 0]

IndexError: index 22 is out of bounds for axis 0 with size 22

# Parameterizing the model

In [1999]:
# basic model parameters
dispersal = True
nMales = 25
nFemales = 25
nGroups = 10; dispersal = False if nGroups < 2 else dispersal
totalAgents = nAgents * nGroups
cycleLength = 30
rankFitnessCorrelation  = 0.8
synchrony = 0.0 # seasonality vs. group-size influences
mutationRate = 0.01
migrationRate = 0.01
maleDispersalBias = 0.5
realTimePlots = False
#whichPlot = "Pairs"
whichPlot = "swelling"
realTimePlotsRate = 0.1
nDays = 10
nGenerations = 40
#swellingFunction = "eachDay"
swellingFunction = "slopes"
numberGenes = cycleLength if swellingFunction == "eachDay" else 5

# cycle parameters
ovulation = 16
prePOPLength = ovulation - 6
postPOPLength = cycleLength - prePOPLength - 6
conceptionProbabilityList = [0] * prePOPLength
conceptionProbabilityList += [.05784435, .16082819, .19820558, .25408223, .24362408, .10373275]
conceptionProbabilityList += [0] * postPOPLength

# swelling parameters
swellingConspicuousness = 1.0 # 0.0 - 1.0 does a less conspicuous swelling simply mean more noise in male preferences?
swellinging = True; swellingConspicuousness = 0.0 if swellinging == False else swellingConspicuousness
swellingSD = 2.5
swellingMean = 15
#startingswelling = "noswelling"
startingSwelling = "randomUniform"

# model mechanics parameters


# Running a model

In [ ]:
nGenerations = 10
nDays = 100
nGroups = 10
dispersal = True
realTimePlots = True
model = evolvingModel()
model.evolve()

## Everything below this point is for testing code

In [ ]:
for s in np.arange(0,1.1,0.2):
    for r in np.arange(0,1.1,0.2):
        rankFitnessCorrelation  = r
        synchrony = s
        nGenerations = 1000
        nDays = 100
        nGroups = 2
        dispersal = True
        realTimePlots = False
        model = evolvingModel()
        model.evolve()
        print(s)

In [ ]:
## run simulations with different parametizations for various variables
models = []
for i in np.arange(1.0,1.1,0.25):
    rankFitnessCorrelation  = i
    #sychrony = i
    #swellingConspicuousness = i
    swellingSD = 1.5
    swellingMean = 14
    scale = swellingConspicuousness
    model = ABM()
    model.setupAgents()
    model.runModel()

    showConceptionFunctionPlot = False
    showSwellingFunctionPlot = False
    showConceptionFunctionAndSwellingPlot = False
    showConceptionFunctionAndAttractivenessPlot = True
    showConceptionRiskAttractivenessCorrelation = False
    showFemaleReproductiveSuccessPlot = True
    conspicuousnessFitnessCorrelation = True
    
    if i == 0.0 and showConceptionFunctionPlot == True:
        plt.plot(range(1, cycleLength + 1), model.conceptionProbabilityList, 'bo')
        plt.ylim(0, 1.2)
        plt.title('Shape of conception probability function', fontsize = 17)
        
    if showSwellingFunctionPlot == True:
        plt.figure()
        plt.plot(range(1, cycleLength + 1), model.swellingList, 'ro')
        plt.xlabel('Shape of swelling function', fontsize=15)
        plt.ylim(0, swellingConspicuousness * 5)
        plt.title('Swelling conspicuousness is ' + str(swellingConspicuousness), fontsize = 17)
    
    if showConceptionFunctionAndSwellingPlot == True:
        plt.figure()
        plt.plot(model.swellingList, 'ro')
        plt.plot(model.conceptionProbabilityList, 'bo')
        plt.ylim(0, 1.2)
        plt.xlabel('Cycle Day', fontsize=15)
        plt.legend(['Swelling', 'Conception Probability'], bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.title('Swelling conspicuousness is ' + str(swellingConspicuousness), fontsize = 17)
    
    if showConceptionFunctionAndAttractivenessPlot == True:
        plt.figure()
        plt.plot(model.dayList,model.dailyAttractivenessList, 'go')
        plt.plot(model.dayList,model.dailyConceptionProbabilityList, 'bo')
        plt.ylim(0, 1.2)
        plt.xlabel('Cycle Day', fontsize=15)
        plt.legend(['Attractiveness', 'Conception Probability'], bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.title('Swelling conspicuousness is ' + str(swellingConspicuousness), fontsize = 17)

    if showConceptionRiskAttractivenessCorrelation == True:
        plt.figure()
        plt.plot(model.dailyConceptionProbabilityList,model.dailyAttractivenessList, 'bo')
        plt.ylim(0, 1)
        plt.xlabel('Conception Probability', fontsize=15)
        plt.ylabel('Attractiveness', fontsize=15)
        #plt.ylim(0, (swellingConspicuousness * 5))
        plt.title('Swelling conspicuousness is ' + str(swellingConspicuousness), fontsize = 17)

    if showFemaleReproductiveSuccessPlot == True:
        reproductiveSuccesss = [f.reproductiveSuccess for f in model.females if f.reproductiveSuccess != False]
        plt.figure(); plt.hist(reproductiveSuccesss, range=[0,1])
        plt.ylabel('Number of females', fontsize=15)
        plt.xlabel('Fitness of mate', fontsize=15)
        plt.title(str(nFemales - len(reproductiveSuccesss)) + '/' + str(nFemales) + ' (' +
                  str(round(100 - len(reproductiveSuccesss) / nFemales * 100, 2)) +'%) did not conceive', fontsize = 17)
        
    if conspicuousnessFitnessCorrelation == True:
        conspicuousness = [f.swellingConspicuousness for f in model.females if f.reproductiveSuccess != False]
        reproductiveSuccesss = [f.reproductiveSuccess for f in model.females if f.reproductiveSuccess != False]
        plt.figure(); plt.plot(conspicuousness, reproductiveSuccesss, 'bo')
        plt.xlabel('Swelling conspicuousness', fontsize=15)
        plt.ylabel('Fitness of mate', fontsize=15)

    model.reset()

In [ ]:
# run a particuar model parameterization multiple times
for i in range(5):
    model.setupAgents()
    model.runModel()
    
    reproductiveSuccesss = [f.reproductiveSuccess for f in model.females if f.reproductiveSuccess != False]
    plt.figure(); plt.hist(reproductiveSuccesss, range=[0,1])
    plt.title(str(nFemales - len(reproductiveSuccesss)) + '/' + str(nFemales) + ' did not conceive')    
    
    model.reset()

In [ ]:
## test various variables
for i in np.arange(0,1.1,0.5):
    #rankFitnessCorrelation  = i
    synchrony = i
    #swellingConspicuousness = i
    model = ABM()
    model.setupAgents()
    #plt.figure(); plt.plot([m.rank for m in model.males], [m.fitness for m in model.males], 'bo')
    plt.figure(); plt.hist([f.cycleDay for f in model.females], range=[1, cycleLength])
    model.reset()

In [ ]:
nDays = 4
ranks = range(nMales)
rankFitnessCorrelation = 1.0
cycleLength = 30
synchrony = 0.0
model = evolvingModel()

model.groups = [group(0)]

cycles = math.ceil(nDays / cycleLength)

IDs = [d for d in range(nFemales)] * nDays
cycleDays = [f.cycleDay-1 for f in model.groups[0].females]

swellings = []
conceptionRisks = []

swellDictList = []

for i in range(nDays):
  swellings += [(f.swellingList  * nDays)[cycleDays[f.ID] + i] for f in model.groups[0].females]
  conceptionRisks += [(conceptionProbabilityList * nDays)[cycleDays[f.ID] + i] for f in model.groups[0].females]

fitnesses = [[m.fitness, m.rank] for m  in model.groups[0].males]
fitnesses = [sorted(fitnesses, key=lambda fitness: fitness[0])]  * nDays

for day in range(nDays):
 swellDictList += [[[ID, swellings[day*nFemales + ID],conceptionRisks[day*nFemales + ID], fitnesses[day][ID][1], fitnesses[day][ID][0]] for ID in IDs[day*nFemales:day*nFemales+nFemales]]]


swellDictList = [sorted(swellDictList[i], key=lambda swelling: swelling[1]) for i in range(nDays)]
swellDictList